In [148]:
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from scipy.stats import randint as sp_randint
from scipy.stats import uniform
from sklearn.model_selection import RandomizedSearchCV


In [149]:

df_ultimate_test = pd.read_table("artificial_test.data", sep=" ", header=None)
df_ultimate_test = df_ultimate_test.drop(df_ultimate_test.columns[500], axis=1) # kolumna 500 do wyrzucenia
df_ultimate_test

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,483,454,513,495,523,469,453,477,506,479,...,455,480,543,259,413,520,485,498,523,510
1,485,508,493,487,478,472,504,476,479,475,...,486,480,535,534,514,452,484,495,548,477
2,483,521,507,475,493,486,421,475,496,483,...,491,476,498,495,508,528,486,465,508,503
3,474,504,576,480,553,483,524,478,483,483,...,521,475,470,463,509,525,479,467,552,517
4,495,474,523,479,495,488,485,476,497,478,...,510,471,522,343,509,520,475,493,506,491
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,493,458,503,478,517,479,472,478,444,477,...,469,475,485,443,517,486,474,489,506,506
596,481,484,481,490,449,481,467,478,469,483,...,506,485,508,599,498,527,481,490,455,451
597,485,485,530,480,444,487,462,475,509,494,...,442,474,502,368,453,482,478,481,484,517
598,477,469,528,485,483,469,482,477,494,476,...,473,476,453,638,471,538,470,490,613,492


In [150]:
df_data_X = pd.read_table("artificial_train.data", sep=" ", header=None)
df_data_X = df_data_X.drop(df_data_X.columns[500], axis=1)
df_data_X

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,485,477,537,479,452,471,491,476,475,473,...,477,481,477,485,511,485,481,479,475,496
1,483,458,460,487,587,475,526,479,485,469,...,463,478,487,338,513,486,483,492,510,517
2,487,542,499,468,448,471,442,478,480,477,...,487,481,492,650,506,501,480,489,499,498
3,480,491,510,485,495,472,417,474,502,476,...,491,480,474,572,454,469,475,482,494,461
4,484,502,528,489,466,481,402,478,487,468,...,488,479,452,435,486,508,481,504,495,511
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,490,505,503,474,463,461,519,476,518,467,...,467,479,449,588,499,506,475,463,507,501
1996,480,475,476,480,495,482,515,479,480,484,...,464,474,473,424,454,570,476,493,465,485
1997,480,517,631,470,485,474,535,476,493,466,...,501,483,479,687,488,488,483,500,523,481
1998,484,481,505,478,542,477,518,477,510,472,...,487,483,526,750,486,529,484,473,527,485


In [151]:
df_data_Y = pd.read_table("artificial_train.labels", sep=" ", header=None)
df_data_Y

,0
0,-1
1,-1
2,-1
3,1
4,1
...,...
1995,1
1996,-1
1997,-1
1998,1


In [152]:
X_train, X_test, y_train, y_test = train_test_split(df_data_X, df_data_Y, test_size=0.33, random_state=4, stratify=df_data_Y)

In [153]:
# Define the pipeline

pipe = Pipeline([
    ('reduce_dim', 'passthrough'),
    ('clf', 'passthrough')
])

models_grid=[
    {
        'clf': [KNeighborsClassifier()],
        'clf__n_neighbors': [3,4,5,6,7],
    },
    {
        'clf': [RandomForestClassifier()],
        'clf__n_estimators': [100, 200, 300],
        'clf__max_depth': [ 8, 9 ,10, 11, 12]
    },
    {
        'clf': [SVC()],
    },
    {
        'clf': [ExtraTreesClassifier(),],
        'clf__n_estimators': [100, 200, 300],
        'clf__max_depth': [ 8, 9 ,10, 11, 12]
    }
]
dimgrid=[
    {
        'reduce_dim': [PCA(svd_solver='full')],
        'reduce_dim__n_components': [4, 5, 6,7, 8, 9, 10, 12 ,15, 20,  30, 40, 50],
    },
]
param_grid=[]
for model in models_grid:
    for dimred in dimgrid:
        param_grid.append({**dimred, **model})

balanced_accuracy_scorer = make_scorer(balanced_accuracy_score)
param_grid


[{'reduce_dim': [PCA(svd_solver='full')],
  'reduce_dim__n_components': [4, 5, 6, 7, 8, 9, 10, 12, 15, 20, 30, 40, 50],
  'clf': [KNeighborsClassifier()],
  'clf__n_neighbors': [3, 4, 5, 6, 7]},
 {'reduce_dim': [PCA(svd_solver='full')],
  'reduce_dim__n_components': [4, 5, 6, 7, 8, 9, 10, 12, 15, 20, 30, 40, 50],
  'clf': [RandomForestClassifier()],
  'clf__n_estimators': [100, 200, 300],
  'clf__max_depth': [8, 9, 10, 11, 12]},
 {'reduce_dim': [PCA(svd_solver='full')],
  'reduce_dim__n_components': [4, 5, 6, 7, 8, 9, 10, 12, 15, 20, 30, 40, 50],
  'clf': [SVC()]},
 {'reduce_dim': [PCA(svd_solver='full')],
  'reduce_dim__n_components': [4, 5, 6, 7, 8, 9, 10, 12, 15, 20, 30, 40, 50],
  'clf': [ExtraTreesClassifier()],
  'clf__n_estimators': [100, 200, 300],
  'clf__max_depth': [8, 9, 10, 11, 12]}]

In [154]:
# Perform the grid search
grid = GridSearchCV(pipe, cv=3, n_jobs=-1, param_grid=param_grid, scoring=balanced_accuracy_scorer, verbose=1)
grid.fit(X_train, y_train)


In [155]:
print(f"best trainig score {grid.best_score_} ,score test {grid.score(X_test, y_test)}")

AttributeError: 'GridSearchCV' object has no attribute 'best_score_'

In [ ]:
grid.best_estimator_

Pipeline(steps=[('reduce_dim', PCA(n_components=5, svd_solver='full')),
                ('clf', KNeighborsClassifier(n_neighbors=7))])

In [ ]:
grid.cv_results_

{'mean_fit_time': array([0.51916583, 0.55499895, 0.49433462, 0.53050113, 0.59733629,
        0.51800156, 0.63416974, 0.65066719, 0.50783269, 0.46183729,
        0.48466516, 0.48716577, 0.66883222, 0.66466316, 0.54716674,
        0.61783187, 0.63766344, 0.55050111, 0.82316875, 0.712164  ,
        0.50550175, 0.51232982, 0.57666453, 0.7668314 , 0.62066674,
        0.59417057, 0.71700025, 0.58166718, 0.54433306, 0.56700118,
        0.60066549, 0.63300157, 0.43899981, 0.70916915, 0.58616654,
        0.44883275, 0.66433342, 0.64383252, 0.55750561, 0.69950334,
        0.58699918, 0.58833361, 0.52967064, 0.51816583, 0.63799906,
        0.5830009 , 0.58250062, 0.56050348, 0.58116245, 0.67433206,
        0.81433312, 0.89916897, 0.80966655, 0.6471653 , 0.66999936,
        0.78050073, 0.46216822, 0.55399831, 0.67466776, 0.48366833,
        0.59800339, 0.53233282, 0.60966674, 0.7056663 , 0.51016672,
        0.88133407, 1.15666461, 0.72300156, 1.07916705, 0.86966713,
        0.94933367, 0.98099979,